In [1]:
from Data.PreProcessing import fetchData
from Data.PreProcessing import fetchLabels
from Algorithms.testingUtil import getScore
import os
from sklearn.linear_model import LogisticRegression

In [2]:
data = []

for fileName in os.listdir('Data/csvData'):
    if fileName.startswith('mar9'):
        f = 'Data/csvData/' + fileName
        data.append(fetchData(f))
        
print(data[4][1000])

[1015.0, 1019.0, 1012.0, 1013.0, 33.0, 101.0, 15.0, 10.0, 59.0]


In [3]:
labels = [];

for point in data:
    labelset = fetchLabels(point, 2020, 3, 9, 48.4634, -123.3117, -8)
    labels.append(labelset)

print(labels[1][1000])

[33.888529982151866, 152.52256378934524]


In [4]:
LR_a = LogisticRegression(penalty='l2', C = 1.0, solver = 'lbfgs', max_iter = 100);
LR_b = LogisticRegression(penalty='l2', C = 1.0, solver = 'lbfgs', max_iter = 100);

train_set = [];
train_labels = [];

alphas = [];
betas = [];

for point in range(len(data)-1):
    for line in data[point]:
        alpha = line[4]
        beta = line[5]
                
        '''The angle of the device works such that when laying flat and aligned with it's marker, it is at (0, 0).
        In our case the marker pointed south. Here, we adjust it so that they corrospond to altitude and azimuth'''

        if(alpha < 90):
            alpha_alt = alpha
            beta_alt = beta + 90
        else:
            alpha_alt = 90 - abs(90 - alpha)
            beta_alt = (360 + (beta - 90))%360
    
        train_set.append([alpha_alt, beta_alt]);
        alphas.append(alpha_alt)
        betas.append(beta_alt)

l_alphas = [];
l_betas = [];       

for point in range(len(labels)-1):
    for line in labels[point]:
        train_labels.append(line) 
        l_alphas.append(line[0])
        l_betas.append(line[1]) 
        
l_alphas = [round(i) for i in l_alphas] 
l_betas = [round(i) for i in l_betas]

LR_a.fit(train_set, l_alphas)
LR_b.fit(train_set, l_betas);

predictions_a = LR_a.predict(train_set);
predictions_b = LR_b.predict(train_set);

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [5]:
#The average number of degrees the angle is off by.
def getAvg(predict, labels):
    dif = 0;
    for point in range(len(predict)):
        dif += abs(predict[point]-labels[point])
    dif = dif/len(predict)
    return dif

a = getAvg(predictions_a, l_alphas);
b = getAvg(predictions_b, l_betas);

print([a, b])

[1.5140248396508562, 4.447393907646203]


In [36]:
predictions = [];
temp_labels = [];
for x in range(len(predictions_a)):
    predictions.append([predictions_a[x], predictions_b[x]])
    temp_labels.append([l_alphas[x], l_betas[x]])

print('ALPHA')
templist = []
for x in range(len(predictions_a)):
    if(abs(predictions_a[x] - l_alphas[x]) > 5):
        if [predictions_a[x], l_alphas[x]] not in templist:
            print([predictions_a[x], l_alphas[x]])
            templist.append([predictions_a[x], l_alphas[x]])

print('BETA')
templist = []
for x in range(len(predictions_b)):
    if(abs(predictions_b[x] - l_betas[x]) > 5):
        if [predictions_b[x], l_betas[x]] not in templist:
            print([predictions_b[x], l_betas[x]])
            templist.append([predictions_b[x], l_betas[x]])
            
score = getScore(predictions, temp_labels, 10)

ALPHA
[34, 28]
[28, 34]
[29, 35]
[27, 34]
[27, 33]
[33, 27]
[34, 27]
BETA
[215, 135]
[206, 135]
[148, 136]
[148, 137]
[148, 138]
[147, 138]
[144, 138]
[145, 138]
[147, 139]
[145, 139]
[137, 143]
[136, 143]
[138, 144]
[137, 144]
[136, 144]
[137, 145]
[138, 145]
[153, 147]
[137, 147]
[136, 147]
[136, 148]
[138, 148]
[137, 148]
[137, 149]
[138, 149]
[136, 153]
[135, 153]
[147, 153]
[145, 153]
[144, 153]
[145, 154]
[147, 154]
[145, 155]
[147, 155]
[144, 155]
[138, 155]
[137, 155]
[148, 155]
[136, 155]
[136, 206]
[135, 206]
[213, 206]
[226, 206]
[214, 206]
[213, 207]
[214, 207]
[226, 207]
[214, 208]
[226, 208]
[225, 208]
[226, 209]
[225, 209]
[226, 210]
[225, 210]
[227, 210]
[226, 211]
[225, 211]
[227, 211]
[226, 212]
[207, 213]
[226, 213]
[227, 213]
[225, 213]
[226, 214]
[207, 214]
[225, 214]
[227, 214]
[225, 215]
[209, 215]
[226, 215]
[227, 215]
[209, 216]
[225, 216]
[226, 216]
[227, 216]
[207, 216]
[209, 217]
[226, 217]
[207, 217]
[211, 217]
[225, 217]
[227, 217]
[136, 224]
[135, 224]
[2

In [7]:
test_labels = [];
test_set = [];
t_alphas = [];
t_betas = [];       

point = len(data) - 1
for line in data[point]:
        alpha = line[4]
        beta = line[5]

        if(alpha < 90):
            alpha_alt = alpha
            beta_alt = beta + 90
        else:
            alpha_alt = 90 - abs(90 - alpha)
            beta_alt = (360 + (beta - 90))%360
    
        test_set.append([alpha_alt, beta_alt]);
        alphas.append(alpha_alt)
        betas.append(beta_alt)

for line in labels[point]:
    test_labels.append(line)
    t_alphas.append(line[0])
    t_betas.append(line[1]) 
        
t_alphas = [round(i) for i in t_alphas] 
t_betas = [round(i) for i in t_betas]

predictions_at = LR_a.predict(test_set);
predictions_bt = LR_b.predict(test_set);

In [8]:
a = getAvg(predictions_at, l_alphas);
b = getAvg(predictions_bt, l_betas);

print([a, b])

[3.2144045771553578, 85.1272740870931]


In [38]:
predictionst = [];
temp_labels = [];
for x in range(len(predictions_at)):
    predictionst.append([predictions_at[x], predictions_bt[x]])
    temp_labels.append([t_alphas[x], t_betas[x]])

print('ALPHA')
templist = []
for x in range(len(predictions_at)):
    if(abs(predictions_at[x] - t_alphas[x]) > 10):
        if [predictions_at[x], t_alphas[x]] not in templist:
            print([predictions_at[x], t_alphas[x]])
            templist.append([predictions_at[x], t_alphas[x]])

print('BETA')
templist = []
for x in range(len(predictions_bt)):
    if(abs(predictions_bt[x] - l_betas[x]) > 10):
        if [predictions_bt[x], l_betas[x]] not in templist:
            print([predictions_bt[x], l_betas[x]])
            templist.append([predictions_bt[x], l_betas[x]])

score = getScore(predictionst, temp_labels, 20)

templist = []
for x in range(len(predictions_bt)):
    if [predictions_bt[x], t_betas[x]] not in templist:
            print([predictions_bt[x], t_betas[x]])
            templist.append([predictions_bt[x], t_betas[x]])

ALPHA
BETA
[206, 135]
[213, 135]
[215, 135]
[225, 135]
[227, 135]
[227, 136]
[225, 136]
[213, 136]
[227, 137]
[225, 137]
[227, 138]
[227, 139]
[225, 139]
[215, 139]
[227, 143]
[225, 143]
[215, 143]
[213, 143]
[227, 144]
[227, 145]
[227, 146]
[227, 147]
Tolerance of 20 degrees
Score: 0.9455714900061701

[136, 228]
[135, 228]
[206, 228]
[213, 228]
[215, 228]
[225, 228]
[227, 228]
[213, 229]
[225, 229]
[227, 229]
[227, 230]
[225, 230]
[227, 231]
[225, 231]
[215, 231]
[227, 232]
[225, 232]
[215, 232]
[213, 232]
[227, 233]
[227, 234]
[227, 235]
[227, 236]
